In [5]:
pip install tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 21.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 21.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 22.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 kB 27.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report



# Load the dataset
train_data = pd.read_csv("/Users/shacheesb/Downloads/jigsaw-toxic-comment-train.csv")

# Preprocessing
train_data = train_data[['comment_text', 'toxic']]
train_data.dropna(inplace=True)

# Text Tokenization
max_words = 20000  # Vocabulary size
max_sequence_length = 200  # Maximum comment length
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['comment_text'])

# Convert texts to sequences
X = tokenizer.texts_to_sequences(train_data['comment_text'])
X = pad_sequences(X, maxlen=max_sequence_length, padding='post')

# Labels
y = train_data['toxic'].values

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Model definition
embedding_dim = 128
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    SpatialDropout1D(0.2),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=False),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 5
batch_size = 64

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

# Evaluate the model
y_pred = (model.predict(X_val) > 0.5).astype(int)
print(classification_report(y_val, y_pred))

# Save the tokenizer and model for later use
model.save("lstm_toxicity_model.h5")
with open("tokenizer.json", "w") as f:
    f.write(tokenizer.to_json())

# Example Prediction
def predict_toxicity(comment):
    sequence = tokenizer.texts_to_sequences([comment])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length, padding='post')
    prediction = model.predict(padded_sequence)[0][0]
    return "Toxic" if prediction > 0.5 else "Non-Toxic"

# Example usage
example_comment = "I hate this so much, you are terrible!"
print(f"Comment: {example_comment}\nPrediction: {predict_toxicity(example_comment)}")


Epoch 1/5


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2795/2795 ━━━━━━━━━━━━━━━━━━━━ 415s 148ms/step - accuracy: 0.9045 - loss: 0.3242 - val_accuracy: 0.9028 - val_loss: 0.2222
Epoch 2/5
2795/2795 ━━━━━━━━━━━━━━━━━━━━ 426s 152ms/step - accuracy: 0.9436 - loss: 0.1509 - val_accuracy: 0.9563 - val_loss: 0.1134
Epoch 3/5
2795/2795 ━━━━━━━━━━━━━━━━━━━━ 418s 149ms/step - accuracy: 0.9592 - loss: 0.1032 - val_accuracy: 0.9567 - val_loss: 0.1109
Epoch 4/5
2795/2795 ━━━━━━━━━━━━━━━━━━━━ 409s 146ms/step - accuracy: 0.9636 - loss: 0.0910 - val_accuracy: 0.9563 - val_loss: 0.1169
Epoch 5/5
2795/2795 ━━━━━━━━━━━━━━━━━━━━ 418s 149ms/step - accuracy: 0.9687 - loss: 0.0778 - val_accuracy: 0.9536 - val_loss: 0.1282
1398/1398 ━━━━━━━━━━━━━━━━━━━━ 47s 33ms/step


              precision    recall  f1-score   support

           0       0.97      0.98      0.97     40444
           1       0.77      0.74      0.75      4266

    accuracy                           0.95     44710
   macro avg       0.87      0.86      0.86     44710
weighted avg       0.95      0.95      0.95     44710

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Comment: I hate this so much, you are terrible!
Prediction: Toxic


ModuleNotFoundError: No module named 'tensorflow'